(This notebook is just being used to document and catalog any Exceptions and Unwanted behaviour encountered when stitching together NumPy & PySyft)

## Still to test:

- Issues encountered when uploading NumPy arrays
- Uploading a project when there are no input assets

In [1]:
import numpy as np

In [2]:
from syft.core.node.new.action_object import ActionObject as AO

In [3]:
data = np.random.rand(5,5)

In [4]:
n = AO.from_obj(data)

In [5]:
type(n)

syft.core.node.new.numpy.NumpyArrayObject

In [6]:
np.asarray(n)

array([[0.94034497, 0.01211849, 0.76530758, 0.95788964, 0.41410558],
       [0.6089372 , 0.72215916, 0.61982241, 0.54588187, 0.47737415],
       [0.64084036, 0.71731214, 0.47828998, 0.79497013, 0.79889449],
       [0.8205206 , 0.3586599 , 0.3678534 , 0.41150896, 0.81681184],
       [0.02904938, 0.21526024, 0.75503014, 0.00230844, 0.56399098]])

# ActionObjects can be cajoled into NumPy arrays without qualms


- Changing dype using np.astype() is fine

In [7]:
type(np.asarray(n))

numpy.ndarray

In [8]:
type(np.max(n))

syft.core.node.new.numpy.NumpyScalarObject

In [9]:
type(np.square(n))

syft.core.node.new.numpy.NumpyArrayObject

In [10]:
type(np.all(n))

syft.core.node.new.numpy.NumpyBoolObject

# Actually can be cajoled into many types:

In [11]:
import pandas as pd

In [13]:
p = pd.DataFrame(n)

In [14]:
type(p)

pandas.core.frame.DataFrame

In [18]:
type(n.astype(np.int64))

syft.core.node.new.numpy.NumpyArrayObject